## Integrantes
- Nelson García  
- Diego Linares
- Joaquin Puente
- José Mérida
- Joaquín Campos

# Análisis exploratorio

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from ydata_profiling import ProfileReport

### Carga de Datos y Revisión General

In [ ]:
# estaremos definiendo ambos csvs para poder tener acceso a ambos pero usaremos el train y luego sobre el test replicaremos una vez se considere importante
dftest = pd.read_csv("../data/test.csv")

# Definir NA como nuestros NaN
dftrain = pd.read_csv("../data/train.csv", na_values=["NA"])

# incluimos en el analissi exploratorio lo basico para poder tenerlo a la mano
dftrain.head()

In [ ]:
# datos estadísticos básicos 
dftrain.describe()

In [ ]:
# tipos
dftrain.info()

In [ ]:
# Revisamos posibles variables redundantes entre sí mismas
print(dftrain.nunique())

## Eliminación de Filas Duplicadas

In [ ]:
before = dftrain.shape[0]

# Eliminar duplicados
dftrain = dftrain.drop_duplicates()
after = dftrain.shape[0]
print(f"Filas eliminadas: {before - after}")

No existen filas duplicadas.

# Exploración de Variables Categóricas

In [ ]:
# cuántas categorías únicas hay por columna?
print(dftrain.select_dtypes(include=['object']).nunique())  

print("Distribucion de categorias por columna: ") 
# Distribución de categorías por columna
for col in dftrain.select_dtypes(include=['object']).columns:
    print(dftrain[col].value_counts())  

Gracias a este output podemos "observar" de forma rápida, que algunas de las variables como utilities, un poco LandSlope, Condition2 y tal vez otras variables pueden ser eliminadas, pero necesitamos poder justificar, de esta forma igual ya nos podemos hacer una idea de como hay algunas variables que tienen poca relevancia. 

# Exploracion de Variables Numéricas

In [ ]:
# Visualizacion y observacion variables numericas
dftrain.hist(figsize=(20, 18), bins=30)
plt.show()

In [ ]:
# Boxplots para detectar outliers
for col in dftrain.select_dtypes(include=['number']).columns:
    sns.boxplot(x=dftrain[col])
    plt.show()

In [ ]:
# correlación evaluacion de posibles variables eliminables
plt.figure(figsize=(26, 20))
df_numeric = dftrain.select_dtypes(include=['number'])
sns.heatmap(df_numeric.corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.show()

### Evaluaciones entre variables 2darias para ver redundancia con la 1


In [ ]:
# condition
condition_diff_count = (dftrain["Condition1"] != dftrain["Condition2"]).sum()
print(f"Number of rows with different Condition1 and Condition2: {condition_diff_count}")
# exterior
exterior_diff_count = (dftrain["Exterior1st"] != dftrain["Exterior2nd"]).sum()
print(f"Number of rows with different Exterior1st and Exterior2nd: {exterior_diff_count}")
# BsmtFinSF2
bsmt_diff_count = (dftrain["BsmtFinSF1"] != dftrain["BsmtFinSF2"]).sum()
print(f"Number of rows with different BsmtFinSF1 and BsmtFinSF2: {bsmt_diff_count}")

En conclusion podemos definir que las variables 2darias no las podemos eliminar del todo porque numeros como 195 son pesados o voluminosos para posibles calculos en el futuro con SalesPrice


### Variables con valores nulos


In [ ]:
# Verificar cuántos valores nulos hay en cada columna
missing_values = dftrain.isnull().sum() / len(dftrain) * 100
print(missing_values.sort_values(ascending=False))

Con este output podemos eliminar 4 variables irrelevantes (PoolQC, MiscFeature, Alley, Fence) porque tienen NaN en 80% o más (recordemos que NA se establecio el NA como el na_value al leer en el csv)

## ¿El Id valdrá la pena?
Al ser el Id solo un identificador unico de las casas no es relevante para el valor que estamos buscando predecir

In [ ]:
dftrain_original = dftrain.copy()  # Guarda una copia por si acaso
dftrain = dftrain.drop(["Id", "MiscFeature", "PoolQC", "Alley", "Fence"], axis=1)

In [ ]:
#Para verificar que se han eliminado.
print(dftrain.columns.tolist())

Despues de eliminar esas columnas, podriamos volver a contar valores nulos por columna. 

In [ ]:
# Contar valores nulos por columna
missing_values = dftrain.isnull().sum().sort_values(ascending=False)
missing_values[missing_values > 0]

Para variables nulos en numericas se reemplazaran con la mediana.

In [ ]:
# Reemplazo en numéricas
num_cols = dftrain.select_dtypes(include=["number"]).columns
dftrain[num_cols] = dftrain[num_cols].fillna(dftrain[num_cols].median())
